In [8]:
import pandas as pd
from scipy.stats import ttest_ind

def analyze_multiple_cities(file_paths, sentiment_col='sentiment', city_col='city_name'):
    """
    여러 도시 파일에서 긍정 비율을 계산하고 도시별 데이터를 통합하여 반환하는 함수.

    Parameters:
    - file_paths (dict): 파일 경로와 도시 이름의 딕셔너리, 예: {"도시A": "city_a.csv", "도시B": "city_b.csv"}
    - sentiment_col (str): 긍정/부정 값이 있는 열 이름 (기본값: 'sentiment')
    - city_col (str): 통합 데이터프레임에 도시명을 저장할 열 이름 (기본값: 'city_name')

    Returns:
    - combined_df (pd.DataFrame): 통합된 데이터프레임 (도시별 긍정 비율 포함)
    - summary (pd.DataFrame): 각 도시별 긍정 리뷰 수, 전체 리뷰 수, 긍정 비율 데이터프레임
    """
    city_data = []

    for city, file_path in file_paths.items():
        df = pd.read_csv(file_path)
        df[sentiment_col] = df[sentiment_col].map({'긍정적': 1, '부정적': 0})
        df[city_col] = city
        city_data.append(df)

    # 통합 데이터프레임 생성
    combined_df = pd.concat(city_data, ignore_index=True)

    # 도시별 긍정 비율 계산
    summary = combined_df.groupby(city_col)[sentiment_col].agg(['sum', 'count'])
    summary.rename(columns={'sum': '긍정 리뷰 수', 'count': '전체 리뷰 수'}, inplace=True)
    summary['긍정 비율 (%)'] = (summary['긍정 리뷰 수'] / summary['전체 리뷰 수']) * 100

    return combined_df, summary



In [10]:
# 도시 파일 경로 설정
file_paths = {
    "경주시": "경주데이터\경주_with_sentiment.csv",
    "강릉시": "강릉데이터\강릉_with_sentiment.csv",
    "당진시": "당진데이터\당진_with_sentiment.csv",
    "충주시": "충주데이터\충주_with_sentiment.csv"
}

# 데이터 분석
combined_df, summary = analyze_multiple_cities(file_paths)

# 그룹 나누기: 경주시와 강릉시 vs 당진시와 충주시
group_1 = combined_df[combined_df['city_name'].isin(['경주시', '강릉시'])]['sentiment']
group_2 = combined_df[combined_df['city_name'].isin(['당진시', '충주시'])]['sentiment']

# t-검정 수행
t_stat, p_value = ttest_ind(group_1, group_2, equal_var=False)

# 결과 출력
print("도시별 긍정 비율 요약:")
print(summary)

print("\nWelch's T-검정 결과:")
print(f"t-통계량: {t_stat:.4f}, p-값: {p_value:.4f}")

# 결과 해석
if p_value < 0.05:
    print("\n결론: 경주시와 강릉시의 긍정 비율이 당진시와 충주시보다 통계적으로 유의미하게 높다.")
else:
    print("\n결론: 경주시와 강릉시의 긍정 비율이 당진시와 충주시보다 유의미하게 높지 않다.")

도시별 긍정 비율 요약:
           긍정 리뷰 수  전체 리뷰 수  긍정 비율 (%)
city_name                             
강릉시            963     1343  71.705138
경주시           1360     1891  71.919619
당진시            171      271  63.099631
충주시            276      460  60.000000

Welch's T-검정 결과:
t-통계량: 5.4225, p-값: 0.0000

결론: 경주시와 강릉시의 긍정 비율이 당진시와 충주시보다 통계적으로 유의미하게 높다.
